 🦠 Amplicon Sequencing Data Analysis with Qiime 2

In [ ]:
!git clone https://github.com/gibbons-lab/isb_course_2022 materials

Cloning into 'materials'...
remote: Enumerating objects: 1131, done.
remote: Counting objects: 100% (320/320), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 1131 (delta 172), reused 286 (delta 153), pack-reused 811
Receiving objects: 100% (1131/1131), 263.58 MiB | 34.92 MiB/s, done.
Resolving deltas: 100% (474/474), done.
Checking out files: 100% (579/579), done.


In [ ]:
%cd materials

/content/materials


In [ ]:
%run setup_qiime2

[19:27:24] 🐍 Downloading miniconda...                                                              ]8;id=79940;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=519894;file:///content/materials/setup_qiime2#56\56]8;;\

[19:27:25] 🐍 Done.                                                                                 ]8;id=3695;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=575724;file:///content/materials/setup_qiime2#62\62]8;;\

           🐍 Installing miniconda...                                                               ]8;id=286556;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=375008;file:///content/materials/setup_qiime2#56\56]8;;\

[19:27:46] 🐍 Installed miniconda to `/usr/local`.                                                  ]8;id=911722;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=543609;file:///content/materials/setup_qiime2#62\62]8;;\

           🔍 Downloading Qiime 2 package list...                                                   ]8;id=966764;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=982529;file:///content/materials/setup_qiime2#56\56]8;;\

...

[19:37:02] 🌲 Done.                                                                                 ]8;id=580093;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=403062;file:///content/materials/setup_qiime2#62\62]8;;\

           📊 Checking that Qiime 2 command line works...                                           ]8;id=74151;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=157134;file:///content/materials/setup_qiime2#56\56]8;;\

[19:37:13] 📊 Qiime 2 command line looks good 🎉                                                    ]8;id=752008;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=629083;file:///content/materials/setup_qiime2#62\62]8;;\

           🧹 Cleaned up unneeded files.                                                            ]8;id=420985;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=733964;file:///content/materials/setup_qiime2#51\51]8;;\

           Everything is A-OK. You can start using Qiime 2 now 👍                                  ]8;id=13489;file:///content/materials/setup_qiime2\setup_qiime2]8;;\:]8;id=743631;file:///content/materials/setup_qiime2#159\159]8;;\

In [ ]:
!qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path data/manifest.tsv \
  --output-path sequences.qza \
  --input-format PairedEndFastqManifestPhred33V2

Imported data/manifest.tsv as PairedEndFastqManifestPhred33V2 to sequences.qza


In [ ]:
!qiime demux summarize --i-data sequences.qza --o-visualization qualities.qzv

You can view the plot by downloading the .qzv file and opening it using http://view.qiime2.org. To download the file click on the folder symbol to the left, open the `materials` folder, and choose download from the dot menu next to the `qualities.qzv` file.

🤔 What do you observe across the read? Where would you truncate the reads?

# Denoising amplicon sequence variants

We will now run the DADA2 plugin which will do 3 things:

1. filter and trim the reads
2. find the most likely set of unique sequences in the sample (ASVs)
3. remove chimeras
4. count the abundances of each ASV


Since this step takes a bit, let's start the process and use the time to
understand what is happening:

In [ ]:
!qiime dada2 denoise-paired \
    --i-demultiplexed-seqs sequences.qza \
    --p-trunc-len-f 150 \
    --p-trunc-len-r 150 \
    --p-n-threads 2 \
    --output-dir dada2 --verbose

---

While this is running let's get back to the presentation. 📚

---

<br>


If this step takes too long or fails, you can also copy the results from the treasure chest with the following command.

In [ ]:
# obscure magic that will only copy if the previous command failed
![ -d dada2 ] || cp -r treasure_chest/dada2 .

Ok, this step ran, but we should also make sure it kind of worked. One good way to tell if the identified ASVs are representative of the sample is to see how many reads were maintained throughout the pipeline. Here, the most common issues and solutions are:

**Large fraction of reads is lost during merging (only paired-end)**

![read overlap](https://gibbons-lab.github.io/isb_course_2022/16S/assets/read_overlap.png)

In order to merge ASVs DADA2 uses an overlap of 12 bases between forward and reverse reads by default. Thus, your reads must allow for sufficient overlap *after* trimming. So if your amplified region is 450bp long and you have 2x250bp reads and you trim the last 30 bases of each read, truncating the length to 220bp, the total length of covered sequence is 2x220 = 440 which is shorter than 450bp so there will be no overlap. To solve this issue trim less of the reads or adjust the `--p-min-overlap` parameters to something lower (but not too low).

<br>

**Most of the reads are lost as chimeric**

![read overlap](https://gibbons-lab.github.io/isb_course_2022/16S/assets/chimera.png)

This is usually an experimental issue as chimeras are introduced during amplification. If you can adjust your PCR, try to run fewer cycles. Chimeras can also be introduced by incorrect merging. If your minimum overlap is too small ASVs may be merged randomly. Possible fixes are to increase the `--p-min-overlap` parameter or run the analysis on the forward reads only (in our empirical observations, chimeras are more likely to be introduced in the joined reads). *However, losing between 5-25% of your reads to chimeras is normal and does not require any adjustments.*

Our denoising stats are contained in an artifact. To convert it to a visualization we can use `qiime metadata tabulate`.

In [ ]:
!qiime metadata tabulate \
    --m-input-file dada2/denoising_stats.qza \
    --o-visualization dada2/denoising-stats.qzv

What proprotion of reads was retained throughout the entire pipeline? Look at the final number of used reads (non-chimeric). What do you observe when comparing those values between samples and how might that affect diversity metrics?

# Phylogeny and ecological diversity metrics

## Building a tree

We can build a phylogenetic tree for our sequences using the following command:

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada2/representative_sequences.qza \
    --output-dir tree

We can create a visualization for the tree using the [empress](https://github.com/biocore/empress) Qiime 2 plugin.

In [ ]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

This looks nice but is not particularly informative as is. The main utility of our tree will be in complementing our diversity analyses. It will tell us which ASVs are more or less related to one another, which will allow us to calculate different kinds of ecological diversity metrics. But what are those?

---

Let's get back to the presentation. 📚

---

<br>

## Alpha and Beta Diversity

![sample sources](https://github.com/Gibbons-Lab/isb_course_2022/raw/main/docs/16S/assets/sample_sources.png)

One of our main goals will be to compare the microbial composition across different populations. QIIME 2 has "run-all" command for diversity analyses. This will

1. Subsample our samples to the same total number of reads (Why?) 
2. Calculate alpha and beta diversity measures
3. Visualize PCoA projections

## Statistical analyses

Let's first have a look at alpha diversity. Can we see a difference in the per-sample diversity across populations? 

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table dada2/table.qza \
    --i-phylogeny tree/rooted_tree.qza \
    --p-sampling-depth 8000 \
    --m-metadata-file data/metadata.tsv \
    --output-dir diversity

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization diversity/alpha_groups.qzv

Now, let's use beta diversity to see how different the samples are from one another. First download `diversity/weighted_unifrac_emperor.qzv` and take a look. Do samples separate based on the population?

We can check whether that separation is 'significant' by using a PERMANOVA test.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file data/metadata.tsv \
    --p-formula "ethnic_group" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

---

Let's get back to the presentation. 📚

---

<br>

# Taxonomy

We will use a Bayes classifier trained on the NCBI Refseq database version 212. However, there are many more databases which can be downloaded from https://docs.qiime2.org/2022.8/data-resources/. For instance, the SILVA database is very popular and probably the largest 16S database available right now.

In [ ]:
!qiime feature-classifier classify-sklearn \
    --i-reads dada2/representative_sequences.qza \
    --i-classifier ncbi-refseq-genus-515f-806r.qza \
    --p-n-jobs 2 \
    --o-classification taxa.qza

Now let's have a look at the relative abundances of the different bacterial taxa we have in each sample:

In [ ]:
!qiime taxa barplot \
    --i-table dada2/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization taxa_barplot.qzv

We can also collapse data on a particular taxonomic rank using the QIIME 2 [taxa plugin](https://docs.qiime2.org/2021.4/plugins/available/taxa/). Why might we want to look at different taxonomic ranks, rather than just looking at ASVs?

In [ ]:
!qiime taxa collapse \
    --i-table treasure_chest/dada2/table.qza \
    --i-taxonomy treasure_chest/taxa.qza \
    --p-level 6 \
    --o-collapsed-table genus.qza

We can export the table and convert it to a .csv file so that we can analyze these data using tools outside of the QIIME 2 environment.

In [ ]:
!qiime tools export \
    --input-path genus.qza \
    --output-path exported
!biom convert -i exported/feature-table.biom -o genus.tsv --to-tsv

Now the data are in a common format and we can use them, for instance, to draw a heatmap using Pandas and Seaborn. No need to understand every bit of code here. This just serves to illustrate that you can get data out of QIIME 2 for custom visualizations.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

abundances = pd.read_table("genus.tsv", skiprows=1, index_col=0)
abundances.index = abundances.index.str.split(";").str[5]       # Use only the genus name
abundances = abundances[~abundances.index.isin(["g__", "__"])]  # remove unclassified genera
abundances = abundances.sample(50)                              # use 50 random genera

# Let's do a centered log-ratio transform: log x_i - log mean(x)
transformed = abundances.apply(
    lambda xs: np.log(xs + 0.5) - np.log(xs.mean() + 0.5),
    axis=0)

sns.clustermap(transformed.T, cmap="magma", xticklabels=True, figsize=(18, 6))

# Exercises

Okay, that's enough time in the back seat. 

It's time to take the wheel 🚗 

Now you can dive into the data 🏊

## Exercise - Plant a Tree

One visualization that we did not spend a lot of time on was the phylogentic tree of our ASVs. Let's change that! We have seen that there are genera that appear in multiple populations in the previous step. But are the organisms in that genus actually the same?

Let's annotate the tree with our taxonomic classifications and abundances. We will use the empress plugin again but this time with the `community-plot` option. I filled in a template of the command for you. Can you figure out what has to go in the empty spaces?

**QUESTIONS:**

1) Are some of the branch lengths on the tree longer than you would expect? Do you notice anything interesting or suspicious about the taxonomic identities of these branches?

2) Can you find examples of phyla that are polyphyletic (i.e. where clusters of ASVs from the same phylum are found in different locations on the tree, showing different commmon ancestors)? What about polyphyletic taxa at lower taxonomic levels, like at the family or genus levels? Why do you think these patterns exist?

3) Use the "Shear Tree" functionality to only visualize the phylogenetic tree of the genus *Prevotella*. What do you observe? Can you figure out which of the unique V4 sequences are found only in one population? What impact might this have on studies trying to link *Prevotella* to human health?

In [ ]:
# This won't run until you fill in the [EMPTY] spots with the right files ;)

!qiime empress community-plot \
    --i-tree [EMPTY] \
    --i-feature-table dada2/table.qza \
    --m-sample-metadata-file d[EMPTY] \
    --m-feature-metadata-file taxa.qza \
    --o-visualization community-tree-viz.qzv

<br><br><br>

---

# 🦠 Space for your own exploration

Feel free to use this space to run some additional analyses. If you rerun all the previous cells in the notebook (`Runtime > Run All`) you should be all set to start working on the previously generated artifacts. Remember that you can get help in the Slack channel and also in the [Qiime 2 documentation](https://docs.qiime2.org).

In [ ]:
# You can add more code cells with the "+ Code" button on the top right
